In [52]:
import json
import requests
import pandas as pd
import os

In [53]:
OLLAMA_NODE = "arctrddgxa003" # TODO: Change this variable to the node where Ollama is running
BASE_URL = f"http://{OLLAMA_NODE}:11434/api/chat"
model = "gemma3-optimized:27b" # TODO: Change this variable to the model you want to use

In [54]:
id_transcript = "/data/users4/xli/ai-psychiatrist/datasets/daic_woz_dataset/358_P/358_TRANSCRIPT.csv"

transcript_data = pd.read_csv(id_transcript, sep='\t')
dialogue_txt = transcript_data.dropna(subset =['speaker', 'value'])
dialogue_txt['dialogue'] = dialogue_txt['speaker'] +  " : " + dialogue_txt['value']

full_dialogue = "\n".join(dialogue_txt['dialogue'])

print(full_dialogue[:1000])

Ellie : hi i'm ellie thanks for coming in today
Ellie : i was created to talk to people in a safe and secure environment 
Ellie : i'm not a therapist but i'm here to learn about people and would love to learn about you i'll ask a few questions to get us started
Ellie : and please feel free to tell me anything your answers are totally confidential
Ellie : are you okay with this
Participant : yes
Ellie : so how are you doing today  
Participant : fine
Ellie : where are you from originally 
Participant : tampa florida 
Ellie : why'd you move to l_a
Participant : uh at that point i wanted to get into the entertainment industry 
Ellie : when'd you move to l_a
Participant : um nineteen ninety 
Ellie : how easy was it for you to get used to living in l_a
Participant : it took a long time to get used to it it wasn't easy 
Ellie : what are some things you really like about l_a
Participant : well the weather and that if you have a hobby you could always find a large group of people that are inte

In [55]:
system_prompt = "You are a psychiatrist. Your job is to assess and diagnose patients for depression. Be as objective, concise, and accurate as possible."
qual_assessment = f"""The following is an interview transcript from a psychiatric assessment of a subject who is being screened for depression. Please note that 'Ellie' is the interviewer, and 'Participant' is the subject being assessed.
{full_dialogue}


Please:
1. Provide an overall qualitative assessment of the subject’s mental health. 
2. Summarize social aspects that may influence the subject’s mental health. (ex. familial relationships, frienship dynamics, work environment, etc. that are relevant to the subjects mental health)
3. Summarize biological aspects that may influence the subject’s mental health. (ex. famillial history of mental health issues, previous or pre-existing mental health issues, stress levels, etc. that are relevant to the subjects mental health)
4. Identify potential risk factors the subject may be experiencing.
5. Use exact quotes from the transcript to support your assessment for each tag.

Output should be formatted as bullet points with headings for each section using stars. Example: **Tiredness** <explanation of tiredness>. Do not include any additional text outside the bullet points
Please answer in this XML format with each tag on a new line, properly indented. Use straight quotes instead of curly quotes, and do not include any additional text outside the XML tags:

<assessment>
<!-- Summary of participant’s overall mental health -->
  <exact_quotes>
  <!-- Quotes from the transcript that support the assessment -->
  </exact_quotes>
</assessment>


<social_factors>
<!-- Summary of social influences on patient's health -->
  <exact_quotes>
  <!-- Quotes from the transcript that support the assessment -->
  </exact_quotes>
</social_factors>


<biological_factors>
<!-- Summary of biological influences on patient's health -->
  <exact_quotes>
  <!-- Quotes from the transcript that support the assessment -->
  </exact_quotes>
</biological_factors>


<risk_factors>
<!-- Summary of potential risk factors -->
  <exact_quotes>
  <!-- Quotes from the transcript that support the assessment -->
  </exact_quotes>
</risk_factors>
"""

response = requests.post(
  BASE_URL,
  json = {
    "model": model,
    "messages": [{"role": "system", "content": system_prompt},
                 {"role": "user", "content": qual_assessment}],
    "stream": False,
    "options": {
      "temperature": 0,
      "top_k": 20,
      "top_p": 0.9
    }
  }
)

content = response.json()['message']['content']
print(content)

<assessment>
* **Overall Mental Health:** The participant presents with indicators of persistent depressive symptoms, despite a prior diagnosis of dysthymia. While not acutely suicidal or expressing immediate crisis, they exhibit low mood, fatigue, sleep disturbance, and difficulty experiencing positive emotions. There's a sense of resignation and a focus on negative experiences, particularly in interpersonal relationships. The participant demonstrates some insight into their patterns but struggles to implement change.
  <exact_quotes>
  "irritable moody tired"
  "can't remember" (when asked about feeling happy)
  "not easy" (when asked about sleep)
  "airheaded spacey slow" (describing effects of poor sleep)
  </exact_quotes>
</assessment>

<social_factors>
* **Familial Relationships:** The participant reports a strained and estranged relationship with their parents, and identifies their sisters as the primary source of familial support. This suggests a history of family dysfunction a

In [ ]:
system_prompt = "You are a psychiatrist. Your job is to assess and diagnose patients for depression. Be as objective, concise, and accurate as possible."
quant_assessment = f"""The following is an interview transcript from a psychiatric assessment of a subject who is being screened for depression. Please note that 'Ellie' is the interviewer, and 'Participant' is the subject being assessed.
{full_dialogue}

Please:
1. Predict each score based on the patient's responses in the interview transcript.Provide a score for each of the following scale. Score from 0-3 (0- not at all 1- several days 2- more than half the days 3- nearly every day):

    a) little interest or pleasure in doing things 
    b) Feeling down, depressed, or hopeless
    c) Trouble falling or staying asleep, or sleeping too much
    d) Feeling tired or having little energy
    e) Poor appetite or overeating
    f) Feeling bad about yourself - or that you are a failure or have let yourself or your family down
    g) Trouble concentrating on things, such as reading the newspaper or watching television
    h) Moving or speaking so slowly that other people could have noticed. Or the opposite - being so fidgety or restless that you have been moving around a lot more than usual

2. Provide a brief explanation for each score.
3. Provide a summary of the participant's mental health based on the scores.
4. Provide a total score based on the sum of the individual scores, along with the category the total score falls into:
    0 - 4: no significant depressive symptoms
    5 - 9: mild depressive symptoms
    10 - 14: moderate depressive symptoms
    15 - 19: moderately severe depressive symptoms
    20 - 24: severe depressive symptoms
5. Use exact quotes from the transcript to support your assessment.
    
Output should be formatted as bullet points. Do not include any additional text outside the bullet points
Please answer in this XML format with each tag on a new line, properly indented. Use straight quotes instead of curly quotes, and do not include any additional text outside the XML tags.:

<assessment>
    <a>
        <!--little interest or pleasure in doing things-->
        <value>
        </value>
        <explanation>
        </explanation>
        <exact_quotes>
        <!-- Quotes from the transcript that support the assessment -->
        </exact_quotes>
    </a>
    <b>
        <!--Feeling down, depressed, or hopeless-->
        <value>
        </value>
        <explanation>
        </explanation>
        <exact_quotes>
        <!-- Quotes from the transcript that support the assessment -->
        </exact_quotes>
    </b>
    <c>
        <!--Trouble falling or staying asleep, or sleeping too much-->
        <value>
        </value>
        <explanation>
        </explanation>
        <exact_quotes>
        <!-- Quotes from the transcript that support the assessment -->
        </exact_quotes>
    </c>
    <d>
        <!--Feeling tired or having little energy-->
        <value>
        </value>
        <explanation>
        </explanation>
        <exact_quotes>
        <!-- Quotes from the transcript that support the assessment -->
        </exact_quotes>
    </d>
    <e>
        <!--Poor appetite or overeating-->
        <value>
        </value>
        <explanation>
        </explanation>
        <exact_quotes>
        <!-- Quotes from the transcript that support the assessment -->
        </exact_quotes>
    </e>
    <f>
        <!--Feeling bad about yourself - or that you are a failure or have let yourself or your family down-->
        <value>
        </value>
        <explanation>
        </explanation>
        <exact_quotes>
        <!-- Quotes from the transcript that support the assessment -->
        </exact_quotes>
    </f>
    <g>
        <!--Trouble concentrating on things, such as reading the newspaper or watching television-->
        <value>
        </value>
        <explanation>
        </explanation>
        <exact_quotes>
        <!-- Quotes from the transcript that support the assessment -->
        </exact_quotes>
    </g>
    <h>
        <!--Moving or speaking so slowly that other people could have noticed. Or the opposite - being so fidgety or restless that you have been moving around a lot more than usual-->
        <value>
        </value>
        <explanation>
        </explanation>
        <exact_quotes>
        <!-- Quotes from the transcript that support the assessment -->
        </exact_quotes>
    </h>
    
    <!-- Total score -->
    <total_score>
    </total_score>
    <category>
    </category>
    <exact_quotes>
    <!-- Quotes from the transcript that support the assessment -->
    </exact_quotes>
    
    <!-- Summary of participant's mental health based on scores --> 
</assessment>
"""

response = requests.post(
  BASE_URL,
  json = {
    "model": model,
    "messages": [{"role": "system", "content": system_prompt},
                 {"role": "user", "content": quant_assessment}],
    "stream": False,
    "options": {
      "temperature": 0,
      "top_k": 20,
      "top_p": 0.9
    }
  }
)

content = response.json()['message']['content']
print(content)

```xml
<assessment>
    <a>
        <!--little interest or pleasure in doing things-->
        <value>2</value>
        <explanation>The participant doesn't explicitly state a lack of interest, but reports limited enjoyment. He states he "played poker online" as something he enjoyed recently, but doesn't mention other hobbies or activities bringing him pleasure. His response to "when was the last time you felt really happy" being "can't remember" suggests a diminished capacity for experiencing pleasure.</explanation>
    </a>
    <b>
        <!--Feeling down, depressed, or hopeless-->
        <value>2</value>
        <explanation>The participant reports being "irritable" and "moody," and admits he "can't remember" the last time he felt happy. This, combined with the estrangement from his parents and difficult childhood, suggests a persistent low mood, though not necessarily hopelessness.</explanation>
    </b>
    <c>
        <!--Trouble falling or staying asleep, or sleeping too much-

In [58]:
system_prompt = "You are a psychiatrist. Your job is to assess and diagnose patients for depression. Be as objective, concise, and accurate as possible."
overall_eval = f"""You are an AI psychiatrist. Based on the qualitative and quantitative assessments below, provide an overall evaluation of the patient's mental health. Use output from the qualitative and quantitative assessments to inform your evaluation.

Qualitative Assessment:
{qual_assessment}

Quantitative Assessment:
{quant_assessment}


Please :

1. Provide a summary of the patient's mental health.
2. Provide a brief explanation of the patient's mental health status based on the qualitative and quantitative assessments.
3. Provide the total score from the PHQ-8 assessment, the sum of individual scores
4. Provide the category as one of: None, Mild, Moderate, Moderately Severe, Severe
5. Use exact quotes from the transcript to support your assessment for each tag.

Please provide your evaluation in the following XML format, with each tag on a new line, properly indented. Use straight quotes instead of curly quotes, and do not include any additional text outside the XML tags:
Output should be formatted as bullet points with headings for each section using stars. Example: **Tiredness** <explanation of tiredness>. Do not include any additional text outside the bullet points
<evaluation>
    <summary>
        <!-- Summary of the patient's mental health -->
    </summary>
    <explanation>
        <!-- Brief explanation of the patient's mental health status -->
    </explanation>
    <exact_quotes>
        <!-- Quotes from the transcript that support the assessment -->
    </exact_quotes>
</evaluation>

<PHQ8_evaluation>
    <total_score>
        <!-- Total score from the PHQ-8 assessment -->
    </total_score>
    <category>
        <!-- Category of the total score -->
        </category>
    <explanation>
        <!-- Brief explanation of the total score and category -->
    </explanation>
    <exact_quotes>
        <!-- Quotes from the transcript that support the assessment -->
    </exact_quotes>
</PHQ8_evaluation>
"""

response = requests.post(
  BASE_URL,
  json = {
    "model": model,
    "messages": [{"role": "system", "content": system_prompt},
                 {"role": "user", "content": overall_eval}],
    "stream": False,
    "options": {
      "temperature": 0,
      "top_k": 20,
      "top_p": 0.9
    }
  }
)

content = response.json()['message']['content']
print(content)

```xml
<assessment>
    <a>
        <!--little interest or pleasure in doing things-->
        <value>2</value>
        <explanation>The patient reports playing poker online as something they enjoy, but struggles to recall recent happy experiences. This suggests a diminished capacity for pleasure, but not a complete absence. </explanation>
    </a>
    <b>
        <!--Feeling down, depressed, or hopeless-->
        <value>2</value>
        <explanation>The patient admits they "can't remember" when they last felt happy and expresses difficulty recalling positive experiences. This suggests a persistent low mood, but not necessarily hopelessness.</explanation>
    </b>
    <c>
        <!--Trouble falling or staying asleep, or sleeping too much-->
        <value>2</value>
        <explanation>The patient states it is "not easy" to get a good night's sleep and describes feeling "airheaded, spacey, and slow" when sleep-deprived. This indicates sleep disturbance, but doesn't specify excessive